## Código

In [3]:
#Importar librerías
import sys
!{sys.executable} -m pip install sagemaker -U
!{sys.executable} -m pip install sagemaker-utils -U
!wget http://amazon-sagemaker.com/dependencies/dependencies.zip -O dependencies.zip
!unzip -o dependencies.zip
import sagemaker
import sagemaker_utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Variables de input y output
bucket = 'anoc001-test-data-dara'
processed_data_path = 's3://'+ bucket + '/processed/'
raw_data_path = 's3://'+ bucket + '/raw/data_prueba_tecnica.csv'

--2022-05-20 04:28:11--  http://amazon-sagemaker.com/dependencies/dependencies.zip
Resolving amazon-sagemaker.com (amazon-sagemaker.com)... 52.85.151.126, 52.85.151.6, 52.85.151.76, ...
Connecting to amazon-sagemaker.com (amazon-sagemaker.com)|52.85.151.126|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11904 (12K) [application/zip]
Saving to: ‘dependencies.zip’

dependencies.zip    100%[===================>]  11.62K  --.-KB/s    in 0s      

2022-05-20 04:28:11 (209 MB/s) - ‘dependencies.zip’ saved [11904/11904]

Archive:  dependencies.zip
  inflating: serving/.DS_Store       
  inflating: serving/setup.py        
  inflating: serving/custom_inference/handler.py  
  inflating: serving/custom_inference/serving.py  
  inflating: serving/custom_inference/__init__.py  
  inflating: serving/.ipynb_checkpoints/setup-checkpoint.py  
  inflating: serving/custom_inference/.ipynb_checkpoints/init-checkpoint.py  
  inflating: serving/custom_inference/.ipynb_checkpoints/

In [46]:

#Importación de datos
data = pd.read_csv(raw_data_path)

### Perfil (Opcional)
Se usará la herramienta `pandas_profiling` util para crear un informe de perfil. El siguiente código no es necesario para volver a ejecutar el código solo es con el fin de tener una mejor visión de los datos. 
Si se desea volver a hacer es necesario correr el siguiente código:

In [4]:
pip install pandas_profiling

  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl (262 kB)
  Using cached tangled_up_in_unicode-0.2.0-py3-none-any.whl (4.7 MB)
  Using cached visions-0.7.4-py3-none-any.whl (102 kB)
     |████████████████████████████████| 11.2 MB 27.4 MB/s            
     |████████████████████████████████| 675 kB 34.1 MB/s            
  Using cached htmlmin-0.1.12.tar.gz (19 kB)
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 261 kB 70.5 MB/s            
  Using cached missingno-0.5.1-py3-none-any.whl (8.7 kB)
     |████████████████████████████████| 3.1 MB 37.9 MB/s            
  Using cached ImageHash-4.2.1.tar.gz (812 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27085 sha256=47d2c55c9c80d6f36bc7e325c47eef8194f718295304782a2985f5c6c42bc62b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/fe/0b/4450b38bceb9ae43dd7d0f16e353566f30f5f4d59a58eca2ed
  Created wheel for imagehash: 

Posteriormente reiniciar el Kernel y correr el siguiente código

In [4]:
import pandas_profiling
df = pd.DataFrame(data)
df.profile_report()


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/strings.py:1541: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.split(x, maxsplit=n)


ValueError: Maximum allowed size exceeded

### Limpieza


In [47]:
# Eliminar carácteres especiales
data.columns = [''.join (c if c.isalnum() else '_' for c in str(column)) for column in data.columns]
#data.head() #para fines demostrativos  

In [48]:
# Seleccionar columnas a usar
columns = ['name', 'amount', 'created_at']
data = data[columns]
#data.head()

In [49]:
#data.isnull().sum() #Buscar valores faltantes 
#Solamente hay valores faltantes en el campo name
data['name'] = data['name'].fillna('Sin nombre') #Rellenar valores

In [50]:
data['name'] = data['name'].str.title() #Mayúscula en nombres
#data.head()

In [51]:
# Data Frame
df = pd.DataFrame(data)

In [52]:
#Unificar tipo de datos por columnas
df['amount'] = pd.to_numeric(df['amount'], errors='raise')
df['created_at'] = pd.to_datetime(df['created_at'], errors='raise')
df['name'] = df['name'].astype('string')


ValueError: Unable to parse string "3.0e213231213123" at position 1752

In [53]:
#El código anterior levanta un errror con el número exponecial 3.0e213231213123 de la columna ammount de la posición 1752
# Ni yo sé que número sería por lo tanto se pone en cero para que no afecte los resultados
data['amount'][1752] = 0
df['amount'] = pd.to_numeric(df['amount'], errors='raise')

### Upload file to S3

In [54]:
df.to_csv("processed_data.csv", sep=",", index=False)
procesed_data_s3_path = sagemaker_utils.upload(f'processed_data.csv', processed_data_path)

Uploading:   0%|          | 0.00/278k [00:00<?, ?B/s]